In [1]:
#%matplotlib widget
import backtrader as bt
import backtrader.analyzers as btanalyzers
import rba_tools as rba
import pandas as pd
import numpy as np
#import matplotlib
from datetime import datetime
from datetime import date
from datetime import timedelta
from backtrader.utils import num2date

In [2]:
class MaCrossStrategy(bt.Strategy):

    def __init__(self):
        ma_fast = bt.ind.SMA(period = 10)
        ma_slow = bt.ind.SMA(period = 50)

        self.crossover = bt.ind.CrossOver(ma_fast, ma_slow)

    def next(self):
        if not self.position:
            if self.crossover > 0:
                self.buy()
        elif self.crossover < 0:
            self.close()

In [3]:
cerebro = bt.Cerebro()

In [4]:
binance = rba.getBinanceExchange()
dataframe = rba.get_DataFrame(['ETH/BTC'], binance, '1/1/18', '12/31/19')
data = bt.feeds.PandasData(dataname=dataframe,
                               # datetime='Date',
                               nocase=True,
                               )
cerebro.adddata(data)

In [5]:
#dataframe2 = rba.get_DataFrame(['LTC/BTC'], binance, '1/1/18', '12/31/19')
#data2 = bt.feeds.PandasData(dataname=dataframe2,
#                               # datetime='Date',
#                               nocase=True,
#                               )
#cerebro.adddata(data2)

In [6]:
#add observers before running strategy but after feed

In [7]:
cerebro.addstrategy(MaCrossStrategy)

0

In [8]:
cerebro.broker.setcash(1000.0)

In [9]:
cerebro.addsizer(bt.sizers.PercentSizer, percents = 10)

In [10]:
#cerebro.addanalyzer(btanalyzers.SharpeRatio, _name = 'sharpe')
#cerebro.addanalyzer(btanalyzers.Transactions, _name = 'trans')
#cerebro.addanalyzer(btanalyzers.Returns, _name = 'returns')
cerebro.addanalyzer(btanalyzers.TradeAnalyzer, _name = 'trades') #good for summary stats on trades
cerebro.addanalyzer(btanalyzers.PositionsValue, _name = 'pos', cash=True) #this is a good one. Shows position values over time
cerebro.addanalyzer(btanalyzers.AnnualReturn, _name = 'annReturns')

In [11]:
back = cerebro.run()

In [12]:
cerebro.broker.getvalue()

971.1047165789386

In [13]:
for x in back[0].analyzers:
    print(x)

In [14]:
mytrades = back[0].analyzers.trades.get_analysis()

In [15]:
def pintdictdict(indict,indent=0):
    indent += 1
    for x in indict:
        print('\n','   ' * indent,x,end='',sep='')
        val = indict[x]
        if isinstance(val,bt.utils.autodict.AutoOrderedDict):
            pintdictdict(val,indent)
        else:
            print('=',val,sep='',end='')

In [16]:
pintdictdict(mytrades)


   total
      total=8
      open=0
      closed=8
   streak
      won
         current=0
         longest=1
      lost
         current=7
         longest=7
   pnl
      gross
         total=-28.89528342106143
         average=-3.6119104276326786
      net
         total=-28.89528342106143
         average=-3.6119104276326786
   won
      total=1
      pnl
         total=17.76226659598281
         average=17.76226659598281
         max=17.76226659598281
   lost
      total=7
      pnl
         total=-46.657550017044244
         average=-6.665364288149178
         max=-10.6839502813943
   long
      total=8
      pnl
         total=-28.89528342106143
         average=-3.6119104276326786
         won
            total=17.76226659598281
            average=17.76226659598281
            max=17.76226659598281
         lost
            total=-46.657550017044244
            average=-6.665364288149178
            max=-10.6839502813943
      won=1
      lost=7
   short
      total=0
      pnl

In [17]:
annReturns = back[0].analyzers.annReturns.get_analysis()
annReturns

OrderedDict([(2018, 0.012901438968685452), (2019, -0.0412643528597445)])

In [18]:
pos = back[0].analyzers.pos.get_analysis()
for d in [date(2019, 2, 17) + timedelta(days=x) for x in range(15)]:
    print(d,'=',pos[d])

2019-02-17 = [0.0, 998.6389322601159]
2019-02-18 = [0.0, 998.6389322601159]
2019-02-19 = [98.1027525271264, 898.7055554860451]
2019-02-20 = [99.9681185481004, 898.7055554860451]
2019-02-21 = [98.88043377655825, 898.7055554860451]
2019-02-22 = [99.36681861297268, 898.7055554860451]
2019-02-23 = [102.38668050939188, 898.7055554860451]
2019-02-24 = [94.34262359946106, 898.7055554860451]
2019-02-25 = [96.24273139446467, 898.7055554860451]
2019-02-26 = [95.58263768790223, 898.7055554860451]
2019-02-27 = [94.51366002545296, 898.7055554860451]
2019-02-28 = [94.83168088003163, 898.7055554860451]
2019-03-01 = [94.57245379688767, 898.7055554860451]
2019-03-02 = [92.72846732916265, 898.7055554860451]
2019-03-03 = [91.64612744593275, 898.7055554860451]


In [19]:
for x in back[0].stats:
    print(x)

In [20]:
datatrades = back[0].stats[1]
broker = back[0].stats[0]

In [21]:
for x in back[0].stats:
    print(type(x))
    print('x.plotinfo.plot =',x.plotinfo.plot)
    print('x.plotinfo.plotskip =',x.plotinfo.plotskip)
    print('x.plotinfo.subplot =',x.plotinfo.subplot)

<class 'backtrader.observers.broker.Broker'>
x.plotinfo.plot = True
x.plotinfo.plotskip = False
x.plotinfo.subplot = True
<class 'backtrader.observers.trades.Trades'>
x.plotinfo.plot = True
x.plotinfo.plotskip = False
x.plotinfo.subplot = True


In [22]:
myobs = back[0].stats[1]
myobs

In [23]:
isinstance(myobs ,bt.observers.trades.DataTrades)

False

In [24]:
ind = myobs

In [25]:
for lineidx in range(ind.size()):
            print('lineidx =',lineidx)
            line = ind.lines[lineidx]
            linealias = ind.lines._getlinealias(lineidx)
            print('linealias =',linealias)
            lineplotinfo = getattr(ind.plotlines, '_%d' % lineidx, None)
            print('ind.plotlines =',ind.plotlines)
            print('lineplotinfo =',lineplotinfo)
            if not lineplotinfo:
                lineplotinfo = getattr(ind.plotlines, linealias, None)
                print('2 lineplotinfo =',lineplotinfo)
            if not lineplotinfo:
                lineplotinfo = AutoInfoClass()
                print('3 lineplotinfo =',lineplotinfo)
            pltmethod = lineplotinfo._get('_method', 'plot')
            print('pltmethod =',pltmethod)
            if pltmethod != 'plot':
                toskip += 1 - lineplotinfo._get('_plotskip', False)

lineidx = 0
linealias = pnlplus
ind.plotlines = <backtrader.metabase.AutoInfoClass_pl_LineSeries_pl_LineIterator_pl_DataAccessor_pl_ObserverBase_pl_Observer_pl_Trades object at 0x000001687CBF1130>
lineplotinfo = None
2 lineplotinfo = <backtrader.metabase.AutoInfoClass_pl_Trades_pnlplus object at 0x000001687CBF1220>
pltmethod = plot
lineidx = 1
linealias = pnlminus
ind.plotlines = <backtrader.metabase.AutoInfoClass_pl_LineSeries_pl_LineIterator_pl_DataAccessor_pl_ObserverBase_pl_Observer_pl_Trades object at 0x000001687CBF1130>
lineplotinfo = None
2 lineplotinfo = <backtrader.metabase.AutoInfoClass_pl_Trades_pnlminus object at 0x000001687CBF1280>
pltmethod = plot


In [26]:
for line in myobs.lines:
    npary = np.frombuffer(line.array)
    print(len(npary))
    print(npary[~np.isnan(npary)])
    print(len(npary[~np.isnan(npary)]))

1460
[17.7622666]
1
1460
[ -4.05046052  -5.15306062  -9.9198132   -8.83242794 -10.68395028
  -2.1572203   -5.86061717]
7


In [27]:
npary = np.frombuffer(myobs.lines[0].array)
npary

array([nan, nan, nan, ..., nan, nan, nan])

In [28]:
npary[~np.isnan(npary)]

array([17.7622666])

In [29]:
myline = myobs.lines[0]
myline

In [30]:
myline.array[0:1000]

array('d', [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 17.76226659598281, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,

In [31]:
data = {'trades': myline.array}
df = pd.DataFrame(data=data)
df

,trades
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
1455,NaN
1456,NaN
1457,NaN
1458,NaN


In [32]:
df.loc[np.isnan(df['trades']) == False]

,trades
156,17.762267


In [33]:
def get_trades_from_cerebro_run(cerebro_run):
    #get trades. Long is 'long' and shorts is 'short'. This is unconfirmed. Need to verify
    array_list = []
    for strat in cerebro_run[0].stats:
        if isinstance(strat, bt.observers.trades.Trades):
            for line in strat.lines:
                pnl_data = np.frombuffer(line.array)
                np.nan_to_num(pnl_data, copy=False, nan=0)
                array_list.append(pnl_data)
    summed_ary = sum(array_list)
    summed_ary[summed_ary == 0] = np.nan
    return summed_ary[:int(len(summed_ary) / 2)] #not sure what the deal is with the double sized array but this seems to work
result = get_trades_from_cerebro_run(back)
print(len(result))
print(result[~np.isnan(result)])

730
[ 17.7622666   -4.05046052  -5.15306062  -9.9198132   -8.83242794
 -10.68395028  -2.1572203   -5.86061717]


In [34]:
count = 0
for x in result:
    if not np.isnan(x):
        print('count =',count,'val =',x)
    count +=1

count = 156 val = 17.76226659598281
count = 163 val = -4.050460518664533
count = 321 val = -5.153060620768923
count = 393 val = -9.91981319643352
count = 448 val = -8.83242793598715
count = 536 val = -10.6839502813943
count = 667 val = -2.1572202959051476
count = 696 val = -5.8606171678906644


In [35]:
result[:int(len(result) / 2)]

array([        nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,      

In [63]:
def get_buy_sell_from_cerebro_run(cerebro_run, trade_type='buy'):
    #get executed buy or sell orders. type can switch between buy and sell
    data = None
    for strat in cerebro_run[0].getobservers():
        if isinstance(strat, bt.observers.buysell.BuySell):
            line = 0 if trade_type == 'buy' else 1
            data = np.frombuffer(strat.lines[line].array)
    return data[:int(len(data) / 2)] #not sure what the deal is with the double sized array but this seems to work
result = get_buy_sell_from_cerebro_run(back)
print(len(result))
print(result[~np.isnan(result)])

730
[0.06463373 0.07682113 0.03186377 0.03290787 0.036051   0.0302592
 0.01995511 0.0211056 ]


In [59]:
x = 0 if True else 1
print(x)

0


In [36]:
def get_broker_cash_from_cerebro_run(cerebro_run):
    data = None
    for strat in cerebro_run[0].stats:
        if isinstance(strat, bt.observers.broker.Broker):
            #assumes we want line 0 which I think is safe for Broker?
            npary = np.frombuffer(strat.lines[0].array)
            data = npary
    return data
result = get_broker_cash_from_cerebro_run(back)
print(result)
print(len(result))

[1000. 1000. 1000. ...   nan   nan   nan]
1460


[ 0.  0.  0. ... nan nan nan]


In [37]:
def get_pos_analysis(cerebro_run):
    for analyzer in cerebro_run[0].analyzers:
        if isinstance(analyzer, btanalyzers.PositionsValue):
            return analyzer.get_analysis().copy()

In [38]:
def get_cash_including_position(cerebro_run):
    #this assumes that PositionsValue analyzer exists with parameter cash=True (this is not the default) and headers=False (this is the default)
    pos_analysis = get_pos_analysis(cerebro_run)
    data = []
    for key, value in pos_analysis.items():
        data.append(sum(value))
    return np.round_(np.array(data),2)
result = get_cash_including_position(back)
result

array([1000.  , 1000.  , 1000.  , 1000.  , 1000.  , 1000.  , 1000.  ,
       1000.  , 1000.  , 1000.  , 1000.  , 1000.  , 1000.  , 1000.  ,
       1000.  , 1000.  , 1000.  , 1000.  , 1000.  , 1000.  , 1000.  ,
       1000.  , 1000.  , 1000.  , 1000.  , 1000.  , 1000.  , 1000.  ,
       1000.  , 1000.  , 1000.  , 1000.  , 1000.  , 1000.  , 1000.  ,
       1000.  , 1000.  , 1000.  , 1000.  , 1000.  , 1000.  , 1000.  ,
       1000.  , 1000.  , 1000.  , 1000.  , 1000.  , 1000.  , 1000.  ,
       1000.  , 1000.  , 1000.  , 1000.  , 1000.  , 1000.  , 1000.  ,
       1000.  , 1000.  , 1000.  , 1000.  , 1000.  , 1000.  , 1000.  ,
       1000.  , 1000.  , 1000.  , 1000.  , 1000.  , 1000.  , 1000.  ,
       1000.  , 1000.  , 1000.  , 1000.  , 1000.  , 1000.  , 1000.  ,
       1000.  , 1000.  , 1000.  , 1000.  , 1000.  , 1000.  , 1000.  ,
       1000.  , 1000.  , 1000.  , 1000.  , 1000.  , 1000.  , 1000.  ,
       1000.  , 1000.  , 1000.  , 1000.  , 1000.  , 1000.  , 1000.  ,
       1000.  , 1000

In [39]:
def get_percent_cash_change(cerebro_run):
    cash_vals = get_cash_including_position(cerebro_run)
    start_val = cash_vals[0]
    return np.round((cash_vals / start_val - 1) * 100,2)
result = get_percent_cash_change(back)
print(result)

[ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.41  0.62  0.74  0.26  0.52  0.63  0.78  0.81  0.69
  0.92  0.97  1.78  1.92  2.25  2.12  1.85  2.03  1.9   1.82  1.87  1.9
  2.38  2.38  2.27  2.48  2.23  2.39  2.48  2.37  2.25  1.85  1.36  1.7
  1.57  1.78  1.46  0.63  1.19  1.14  1.38  1.35  1.4   1.83  1.63  1.78
  1.78  1.78  1.78  1.83  1.63  1.6   1.37  1.37  1.37  1.37  1.37  1.3

In [40]:
result

array([ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.41,  0.62,  0.74,  0.26,  0.52,  0.63,
        0.78,  0.81,  0.69,  0.92,  0.97,  1.78,  1.92,  2.25,  2.12,
        1.85,  2.03,

In [41]:
import rba_tools.backtest.rba_backtrader_set as rbs

In [42]:
rbs.get_datetime_array(back)[0:10]

array([datetime.datetime(2018, 1, 1, 0, 0),
       datetime.datetime(2018, 1, 2, 0, 0),
       datetime.datetime(2018, 1, 3, 0, 0),
       datetime.datetime(2018, 1, 4, 0, 0),
       datetime.datetime(2018, 1, 5, 0, 0),
       datetime.datetime(2018, 1, 6, 0, 0),
       datetime.datetime(2018, 1, 7, 0, 0),
       datetime.datetime(2018, 1, 8, 0, 0),
       datetime.datetime(2018, 1, 9, 0, 0),
       datetime.datetime(2018, 1, 10, 0, 0)], dtype=object)

In [65]:
def summarize_cerebro_run(cerebro_run):
    #get pandas dataframe of summarized data
    index = rbs.get_datetime_array(cerebro_run)
    data = { 'cash' : rbs.get_cash_including_position(cerebro_run),
             'percent_change' : rbs.get_percent_cash_change(cerebro_run),
             'trades' : rbs.get_trades_from_cerebro_run(cerebro_run),
             'buy' : get_buy_sell_from_cerebro_run(cerebro_run, trade_type='buy'),
             'sell' : get_buy_sell_from_cerebro_run(cerebro_run, trade_type='sell')
            }
    return pd.DataFrame(data=data, index=index)
df = summarize_cerebro_run(back)
df

,cash,percent_change,trades,buy,sell
2018-01-01,1000.0,0.00,NaN,NaN,NaN
2018-01-02,1000.0,0.00,NaN,NaN,NaN
2018-01-03,1000.0,0.00,NaN,NaN,NaN
2018-01-04,1000.0,0.00,NaN,NaN,NaN
2018-01-05,1000.0,0.00,NaN,NaN,NaN
...,...,...,...,...,...
2019-12-27,971.1,-2.89,NaN,NaN,NaN
2019-12-28,971.1,-2.89,NaN,NaN,NaN
2019-12-29,971.1,-2.89,NaN,NaN,NaN
2019-12-30,971.1,-2.89,NaN,NaN,NaN


In [66]:
df['trades']

2018-01-01   NaN
2018-01-02   NaN
2018-01-03   NaN
2018-01-04   NaN
2018-01-05   NaN
              ..
2019-12-27   NaN
2019-12-28   NaN
2019-12-29   NaN
2019-12-30   NaN
2019-12-31   NaN
Name: trades, Length: 730, dtype: float64

In [67]:
df.loc['2018-06-01':'2018-06-30',:]

,cash,percent_change,trades,buy,sell
2018-06-01,1013.47,1.35,NaN,NaN,NaN
2018-06-02,1014.01,1.40,NaN,NaN,NaN
2018-06-03,1018.26,1.83,NaN,NaN,NaN
2018-06-04,1016.33,1.63,NaN,NaN,NaN
2018-06-05,1017.76,1.78,NaN,NaN,NaN
2018-06-06,1017.76,1.78,17.762267,NaN,0.081301
2018-06-07,1017.76,1.78,NaN,NaN,NaN
2018-06-08,1017.83,1.78,NaN,0.076821,NaN
2018-06-09,1018.30,1.83,NaN,NaN,NaN
2018-06-10,1016.27,1.63,NaN,NaN,NaN


In [46]:
strats = cerebro.runstrats[0]
strats

In [47]:
for si, strat in enumerate(strats):
    print('si =',si)
    print('strat =',strat,'\n')

strat

si = 0
strat = <__main__.MaCrossStrategy object at 0x000001687CADF790> 



In [48]:
strat.getobservers()

In [49]:
buysell = strat.getobservers()[1]
buysell

In [57]:
for line in buysell.lines:
    print(line.array)
    print(len(line.array))

array('d', [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.06463373, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.076821135, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan

In [56]:
back[0].getobservers()